In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import math
import random
import datetime as dt
import tensorflow as tf
from collections import deque
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from keras.regularizers import l1, l2, l1_l2

In [ ]:

from google.colab import files
from google.colab import dri

In [ ]:
dri.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Datapath1 = "/content/drive/MyDrive/datav/aex1v1.json"
Datapath2 = "/content/drive/MyDrive/datav/rmsex1v1.json"
Datapath3 = "/content/drive/MyDrive/datav/zcrx1v1.json"
Datapath4 = "/content/drive/MyDrive/datav/sfx1v1.json"
Datapath5 = "/content/drive/MyDrive/datav/sbx1v1.json"
Datapath6 = "/content/drive/MyDrive/datav/scx1v1.json"
Datapath7 = "/content/drive/MyDrive/datav/melx1v1.json"
Datapath8 = "/content/drive/MyDrive/datav/mellogx1v1.json"
Datapath9 = "/content/drive/MyDrive/datav/mfccx1v1.json"
Datapath11 = "/content/drive/MyDrive/datav/stftx1v3.json"
Datapath12 = "/content/drive/MyDrive/datav/berx1v1.json"
Datapath13 = "/content/drive/MyDrive/datav/gfccx1v3.json"
Datapath14 = "/content/drive/MyDrive/datav/gmccx1v3.json"
Datapath15 = "/content/drive/MyDrive/datav/nmfstftx1v2.json"

In [ ]:
# with open(Datapath1, "r") as f1:
#         data1 = json.load(f1)
# with open(Datapath2, "r") as f2:
#        data2 = json.load(f2)
# with open(Datapath3, "r") as f3:
#        data3 = json.load(f3)
# with open(Datapath4, "r") as f4:
#         data4 = json.load(f4)
# with open(Datapath5, "r") as f5:
#        data5 = json.load(f5)
# with open(Datapath6, "r") as f6:
#         data6 = json.load(f6)
# with open(Datapath7, "r") as f7:
#         data7 = json.load(f7)
# with open(Datapath8, "r") as f8:
#       data8 = json.load(f8)
# with open(Datapath9, "r") as f9:
#        data9 = json.load(f9)
# with open(Datapath11, "r") as f11:
#       data11 = json.load(f11)
# with open(Datapath12, "r") as f12:
#           data12 = json.load(f12)
#with open(Datapath13, "r") as f13:
         # data13 = json.load(f13)
#with open(Datapath14, "r") as f14:
#         data14 = json.load(f14)
#with open(Datapath15, "r") as f15:
#        data15 = json.load(f15)


In [ ]:
# replace each inpt according to data set
x = np.array(data15["sparse"])
y = np.array(data15["labels"])

In [ ]:
# create train, validation and test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train,y_train, test_size=0.25)

In [ ]:
# only use this if rehape 2d data into 3d
# Calculate the appropriate second dimension based on the original size and other dimensions
second_dim = X_train.size // (X_train.shape[0] * 1)

# Reshape the data for the LSTM model
X_train = X_train.reshape(X_train.shape[0], second_dim, 1)
X_test = X_test.reshape(X_test.shape[0], second_dim, 1)
X_validation = X_validation.reshape(X_validation.shape[0], second_dim, 1)

# Verify the new shape
print("Shape of X_train after reshape:", X_train.shape)

Shape of X_train after reshape: (1920, 1292, 1)


In [ ]:
def build_model(input_shape):

    # build lstm
    model = keras.Sequential()

    model.add(keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64,return_sequences=True))
    model.add(keras.layers.LSTM(32))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

In [ ]:
    input_shape = (X_train.shape[1], X_train.shape[2]) # 130, 11
    model = build_model(input_shape)

    # compile model
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor (usually validation loss)
    patience=15,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # Verbosity mode (0 = silent, 1 = progress bar)
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)
    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=48, epochs=200)#, callbacks=[early_stopping])
    # evaluate model on test set
    tst = model.evaluate(X_test, y_test, verbose=2)
    test_loss, test_acc =  model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 1292, 128)           │          67,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 1292, 64)            │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 133,418 (521.16 KB)

 Trainable params: 133,418 (521.16 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 14s 116ms/step - accuracy: 0.1446 - loss: 2.2978 - val_accuracy: 0.1817 - val_loss: 2.2664
Epoch 2/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 147ms/step - accuracy: 0.1999 - loss: 2.2455 - val_accuracy: 0.1917 - val_loss: 2.1746
Epoch 3/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 8s 107ms/step - accuracy: 0.1895 - loss: 2.1240 - val_accuracy: 0.2150 - val_loss: 1.9133
Epoch 4/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 106ms/step - accuracy: 0.1893 - loss: 1.9221 - val_accuracy: 0.2150 - val_loss: 1.7818
Epoch 5/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 11s 115ms/step - accuracy: 0.1959 - loss: 1.8732 - val_accuracy: 0.2150 - val_loss: 1.7347
Epoch 6/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 11s 126ms/step - accuracy: 0.2045 - loss: 1.8223 - val_accuracy: 0.2150 - val_loss: 1.7120
Epoch 7/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.1825 - loss: 1.8283 - val_accuracy: 0.2150 - val_loss: 1.7056
Epoch 8/150
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 107ms/step - accuracy: 0.2095 - loss: 1.7815 - val

In [ ]:
#save my model along with date also with name and save it in google drive
import os
from datetime import datetime
# Get current date and time
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
# Define model name
model_name = "nmfstft1601"
# Create directory for model files
model_dir = os.path.join("/content/drive/MyDrive/modelv", model_name)
os.makedirs(model_dir, exist_ok=True)
# Save model files with date in the filename
model.save(os.path.join(model_dir, f"{model_name}_{current_date}.keras"))
# Save model files with date in the filename
model.save(os.path.join(model_dir, f"{model_name}_{current_date}.h5"))
# Print confirmation message
print(f"Model saved to: {model_dir}/{model_name}_{current_date}.keras")

NameError: name 'model' is not defined

In [ ]:
# Pblot the structure of the contructed model.
plot_model(model, to_file = 'model_structure_plot.png', show_shapes = True, show_layer_names = True)

## Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Accuracy

In [ ]:
# accuracy on training data
X_train_prediction = model.predict(X_train)

# Convert predictions to class labels
import numpy as np
X_train_prediction_classes = np.argmax(X_train_prediction, axis=1)

training_data_accuracy = accuracy_score(y_train, X_train_prediction_classes) # Use predicted class labels
print(training_data_accuracy)
print('Accuracy on Training data : ', round(training_data_accuracy*100, 2), '%')

NameError: name 'model' is not defined

In [ ]:
# accuracy on test data
X_test_prediction = model.predict(X_test)

# Conve|rt predictions to class labels
import numpy as np
X_test_prediction_classes = np.argmax(X_test_prediction, axis=1)

test_data_accuracy = accuracy_score(y_test, X_test_prediction_classes) # Use predicted class labels
print(test_data_accuracy)
print('Accuracy on Test data : ', round(test_data_accuracy*100, 2), '%')

### precision

In [ ]:
# precision for training data predictions
# Convert predictions to class labels for training data
X_train_prediction_classes = np.argmax(X_train_prediction, axis=1)

# Calculate precision using predicted class labels
# Set average='macro' to calculate precision for multiclass
precision_train = precision_score(y_train, X_train_prediction_classes, average='macro')
print('Training data Precision =', precision_train)
print('Training data Precision : ', round(precision_train*100, 2), '%')

NameError: name 'np' is not defined

In [ ]:
# precision for test data predictions
# precision for test data predictions
# Convert predictions to class labels for test data
X_test_prediction_classes = np.argmax(X_test_prediction, axis=1)

precision_test = precision_score(y_test, X_test_prediction_classes, average='macro')
print('Test data Precision =', precision_test)
print('Test data Precision : ', round(precision_test*100, 2), '%')

NameError: name 'np' is not defined

### Recall

In [ ]:
# recall for training data predictions
# Assuming X_train_prediction contains probabilities, convert them to class labels
from sklearn.preprocessing import LabelEncoder

# Fit label encoder on training labels (assuming y_train contains class labels)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

# Transform predictions to class labels
X_train_prediction_classes = label_encoder.transform(np.argmax(X_train_prediction, axis=1))

# Calculate recall score using class labels
recall_train = recall_score(y_train, X_train_prediction_classes, average='macro')
print('Training data Recall =', recall_train)
print('Training data Recall : ', round(recall_train*100, 2), '%')

In [ ]:
# recall for test data predictions
# recall for test data predictions
# Assuming X_test_prediction contains probabilities, convert them to class labels
X_test_prediction_classes = np.argmax(X_test_prediction, axis=1)

# Calculate recall score using class labels
recall_test = recall_score(y_test, X_test_prediction_classes, average='macro')
print('Test data Recall =', recall_test)
print('Test data Recall : ', round(recall_test*100, 2), '%')

### F1 score

In [ ]:
# F1 score for training data predictions
f1_score_train = f1_score(y_train, X_train_prediction_classes,average='macro')
print('Training data F1 Score =', f1_score_train)
print('Training data F1 Score : ', round(f1_score_train*100, 2), '%')

In [ ]:
# F1 Score for test data predictions
f1_score_test = recall_score(y_test, X_test_prediction_classes,average='macro')
print('Test data F1 Score =', f1_score_test)
print('Test data F1 Score : ', round(f1_score_test*100, 2), '%')